<a target="_blank" href="https://colab.research.google.com/github/DerOeko/PPO_PyTorch/blob/main/minimalPPO.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Following this tutorial:

https://github.com/bentrevett/pytorch-rl/blob/master/5%20-%20Proximal%20Policy%20Optimization%20(PPO)%20%5BCartPole%5D.ipynb

In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as distributions

import matplotlib.pyplot as plt
import numpy as np

env_name = "CartPole-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)
SEED = 1234

train_env.action_space.seed(SEED)
test_env.action_space.seed(SEED+1)
np.random.seed(SEED)
torch.manual_seed(SEED)

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout = 0.5):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dim, hidden_dim)
        self.fc_2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        x = self.fc_1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc_2(x)
        return x

class ActorCritic(nn.Module):
    def __init__(self, actor, critic):
        super().__init__()
        
        self.actor = actor
        self.critic = critic
        
    def forward(self, state):
        action_pred = self.actor(state)
        value_pred = self.critic(state)
        
        return action_pred, value_pred
    
INPUT_DIM = train_env.observation_space.shape[0]
HIDDEN_DIM = 128
OUTPUT_DIM = train_env.action_space.n

actor = MLP(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
critic = MLP(INPUT_DIM, HIDDEN_DIM, 1)

policy = ActorCritic(actor, critic)

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)
        
policy.apply(init_weights)

ActorCritic(
  (actor): MLP(
    (fc_1): Linear(in_features=4, out_features=128, bias=True)
    (fc_2): Linear(in_features=128, out_features=2, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (critic): MLP(
    (fc_1): Linear(in_features=4, out_features=128, bias=True)
    (fc_2): Linear(in_features=128, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [2]:
LEARNING_RATE = 0.01
MAX_EPISODES = 500
DISCOUNT_FACTOR = 0.99
N_TRIALS = 25
REWARD_THRESHOLD = 475
PRINT_EVERY = 10
PPO_STEP = 5
PPO_CLIP = 0.2
optimizer = optim.Adam(policy.parameters(), lr = LEARNING_RATE)

In [3]:
def train(env, policy, optimizer, discount_factor, ppo_steps, ppo_clip):
        
    policy.train()
        
    states = []
    actions = []
    log_prob_actions = []
    values = []
    rewards = []
    terminated = False
    episode_reward = 0

    state, _ = env.reset()

    while not terminated:
        state = torch.FloatTensor(state).unsqueeze(0)
        
        states.append(state)
        
        action_pred, value_pred = policy(state)
        
        action_prob = F.softmax(action_pred, dim=-1)
        
        dist = distributions.Categorical(action_prob)
        
        action = dist.sample()
        log_prob_action = dist.log_prob(action)
        state, reward, terminated, _, _ = env.step(action.item())
        
        actions.append(action)
        log_prob_actions.append(log_prob_action)
        values.append(value_pred)
        rewards.append(reward)
        
        episode_reward += reward
    
    states = torch.cat(states)
    print(f"States shape: {states.shape}")
    actions = torch.cat(actions)
    print(f"Actions shape: {actions.shape}")
    log_prob_actions = torch.cat(log_prob_actions)
    print(f"log_prob_actions shape: {log_prob_actions.shape}")

    values = torch.cat(values).squeeze(-1)
    print(f"Values shape: {values.shape}")

    returns = calculate_returns(rewards, discount_factor)
    advantages = calculate_advantages(returns, values)
    policy_loss, value_loss = update_policy(policy, 
                                            states, 
                                            actions, 
                                            log_prob_actions, 
                                            advantages, 
                                            returns, 
                                            optimizer, 
                                            ppo_steps, 
                                            ppo_clip)
    return policy_loss, value_loss, episode_reward


In [4]:
def calculate_returns(rewards, discount_factor, normalize = True):
    returns = []
    R = 0
    
    for r in reversed(rewards):
        R = r + R * discount_factor
        returns.insert(0, R)
    returns = torch.tensor(returns)
    
    if normalize:
        returns = (returns - returns.mean())/returns.std()
    
    return returns

In [5]:
def calculate_advantages(returns, values, normalize=True):
    advantages = returns-values
    
    if normalize:
        advantages = (advantages-advantages.mean())/advantages.std()
        
    return advantages

In [6]:
def update_policy(policy, 
                    states, 
                    actions, 
                    log_prob_actions, 
                    advantages, 
                    returns, 
                    optimizer, 
                    ppo_steps, 
                    ppo_clip):
    total_policy_loss = 0
    total_value_loss = 0
    
    advantages = advantages.detach()
    log_prob_actions = log_prob_actions.detach()
    actions = actions.detach()
    
    for _ in range(ppo_steps):
        action_pred, value_pred = policy(states)
        value_pred = value_pred.squeeze(-1)
        action_prob = F.softmax(action_pred, dim = -1)
        dist = distributions.Categorical(action_prob)

        new_log_prob_actions = dist.log_prob(actions)
        
        policy_ratio = (new_log_prob_actions - log_prob_actions).exp()
        
        policy_loss_1 = policy_ratio * advantages
        policy_loss_2 = torch.clamp(policy_ratio, min = 1.0 - ppo_clip, max = 1.0 + ppo_clip)* advantages
        policy_loss = -torch.min(policy_loss_1, policy_loss_2).sum()
        
        value_loss = F.smooth_l1_loss(returns, value_pred).sum()
        
        optimizer.zero_grad()
        
        policy_loss.backward()
        value_loss.backward()
        
        optimizer.step()
        
        total_policy_loss += policy_loss.item()
        total_value_loss += value_loss.item()
    return total_policy_loss/ppo_steps, total_value_loss/ppo_steps

In [7]:
def evaluate(env, policy):
    policy.eval()
    
    rewards = []
    terminated = []
    episode_reward = 0
    
    state, _ = env.reset()
    
    while not terminated:
        
        state = torch.FloatTensor(state).unsqueeze(0)
        
        with torch.no_grad():
            action_pred, _ = policy(state)
            action_prob = F.softmax(action_pred, dim= -1)
        action = torch.argmax (action_prob, dim = -1)
        state, reward, done, _, _ = env.step(action.item())
        
        episode_reward += reward
    return episode_reward


In [8]:
train_rewards = []
test_rewards = []

for episode in range(1, MAX_EPISODES+1):
    policy_loss, value_loss, train_reward = train(train_env, policy, optimizer,
                                                  DISCOUNT_FACTOR, PPO_STEP,
                                                  PPO_CLIP)
    
    test_reward = evaluate(test_env, policy)
    
    train_rewards.append(train_reward)
    test_rewards.append(test_reward)
    
    mean_train_rewards = np.mean(train_reward[-N_TRIALS:])
    mean_test_rewards = np.mean(test_rewards[-N_TRIALS:])
    
    if episode % PRINT_EVERY == 0:
        print(f'| Episode: {episode:3} | Mean Train Rewards: {mean_train_rewards:5.1f} | Mean Test Rewards: {mean_test_rewards:5.1f} |')
    if mean_test_rewards >= REWARD_THRESHOLD:
        print(f'Reached reward threshold in {episode} episodes')
        
        break

plt.figure(figsize=(12,8))
plt.plot(test_rewards, label = "Test Rewards")
plt.plot(train_rewards, label ="Train Reward")
plt.xlabel("Episodes", fontsize = 20)
plt.ylabel("Reward", fontsize = 20)
plt.hlines(REWARD_THRESHOLD, 0, len(test_rewards), color = "r")
plt.legend(loc="lower right")
plt.grid()

/home/alleinzell/mambaforge/envs/PPOenv/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/alleinzell/mambaforge/envs/PPOenv/lib/python3.12/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


States shape: torch.Size([25, 4])
Actions shape: torch.Size([25])
log_prob_actions shape: torch.Size([25])
Values shape: torch.Size([25])


KeyboardInterrupt: 

In [ ]:
state = torch.FloatTensor(np.array([-0.01557645, -0.04729956,  0.02953374,  0.01043142])).unsqueeze(0)
policy(state)

(tensor([[-0.0020,  0.0101]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0061]], grad_fn=<AddmmBackward0>))